In [2]:
import math

import matplotlib.pyplot as plt
import numpy as np
import os
import time

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import qiskit as qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, transpile, Aer, IBMQ

import ramsey_experiment
import estimator

# Loading your IBM Quantum account(s)
IBMQ.save_account(os.environ.get('IBM_KEY'), overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research')
import Ramsey_ExperimentV2
import random
import importlib
from tqdm import tqdm
ramsey_experiment = importlib.reload(Ramsey_ExperimentV2)
RamseyExperiment = ramsey_experiment.RamseyExperiment
RamseyBatch = ramsey_experiment.RamseyBatch

ibmqfactory.load_account:WARNING:2024-04-07 12:25:59,148: Credentials are already in use. The existing account in the session will be replaced.


In [7]:
import numpy as np
import math
n = 5
total_shots = 10000
T = np.pi
measurements = 50
delay = np.linspace(0, T, measurements)

W = [random.gauss(0, 0) for _ in range(n)]
L = [random.gauss(3, 2) for _ in range(n)]
J = [random.gauss(0, 3) for _ in range(n - 1)]

# L = [3,3,3]
# J = [2,2]
# L = [0,0,0]
# W = [4,4,4]


delay = [0.3]
batch_x, batch_y = ramsey_experiment.ramsey_global(n,total_shots,delay,L,W,J)
rot = 0.23
batch_x_rot, batch_y_rot = ramsey_experiment.ramsey_global_with_rot(n,total_shots,delay,L,W,J, rot)
batch_x_det, batch_y_det, batch_x_cross, batch_y_cross = ramsey_experiment.ramsey_local(n,total_shots,delay,L,W,J)


In [4]:
ramsey_experiment = importlib.reload(Ramsey_ExperimentV2)
RamseyExperiment = ramsey_experiment.RamseyExperiment
RamseyBatch = ramsey_experiment.RamseyBatch
estimator = importlib.reload(estimator)

## qubit by qubit


In [8]:

guessed_decay,guessed_W,guessed_J = estimator.one_by_one_fit(batch_x_det, batch_y_det, batch_x_cross, batch_y_cross)
print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,W,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)

guessed decay:  [0.3243093969076287, -5.226584713517405e-05, -0.0008968253346778871, 4.174910175209545, 2.5417789211136017]
real decay:  [0.3007170650651241, -0.04706722649170203, -1.12457402013262, 3.950294420845772, 2.6334609116457415]
guessed frequency:  [-0.04702803762633055, -0.018666471541449223, 0.07731946318122916, 0.5719709807035781, -0.06288562007223597]
real frequency:  [0.0, 0.0, 0.0, 0.0, 0.0]
guessed J:  [-1.6237081740624586, -1.2310651677482003, 1.749546359760401, 1.2756431865781481]
real J:  [-1.7270905000975207, -1.1177742488415157, 1.7246066827535884, 1.171962245423118]
error:  0.3483444012950952
error in percent:  35.345569067945085


## global

In [9]:
neighbors = 3
guessed_decay,guessed_W,guessed_J = estimator.full_complex_fit(batch_x, batch_y, neighbors=neighbors, W_given=W)

print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)
print("number of neighbors: ", neighbors)

guessed decay:  [3.14350624e-01 2.00108216e-10 3.76151787e-17 3.70947346e+00
 2.72940467e+00]
real decay:  [0.3007170650651241, -0.04706722649170203, -1.12457402013262, 3.950294420845772, 2.6334609116457415]
guessed frequency:  [-1.77503384 -1.11605681  1.78866684  1.06568863]
real frequency:  [0.0, 0.0, 0.0, 0.0, 0.0]
guessed J:  []
real J:  [-1.7270905000975207, -1.1177742488415157, 1.7246066827535884, 1.171962245423118]
error:  0.3875804093566552
error in percent:  25.281476504347815
number of neighbors:  3


## global Noise

In [42]:

neighbors = 0
guessed_decay,guessed_W, guessed_J = estimator.full_complex_fit_test(batch_x_rot, batch_y_rot, batch_x, batch_y, rot1=rot)

print("guessed decay: ", guessed_decay)
print("real decay: ", L)
print("guessed frequency: ", guessed_W)
print("real frequency: ", W)
print("guessed J: ", guessed_J)
print("real J: ", J)

guessed = np.concatenate([guessed_decay,guessed_W,guessed_J])
real_parameters = np.concatenate([L,W,J])
mse = np.mean((guessed - real_parameters)**2)
print("error: ", np.sqrt(mse))
print("error in percent: ", np.sqrt(mse)/np.mean(np.abs(real_parameters)) * 100)
print("number of neighbors: ", neighbors)

TypeError: can't multiply sequence by non-int of type 'numpy.float64'

In [40]:
import Symbolic.symbolic_evolution as symbolic_evolution
importlib.reload(symbolic_evolution)
symbolic_exp = symbolic_evolution.get_expectation_values_exp(n, neighbors=neighbors)


In [42]:
import matplotlib.pyplot as plt

data = []
for i in range(len(delay)):
    data.append(batch_x.RamseyExperiments[i].get_n_nearest_neighbors(neighbors))
    data.append(batch_y.RamseyExperiments[i].get_n_nearest_neighbors(neighbors))
data = np.concatenate(data)

model = []
guessed = []
for t in delay:
    for i in range(2*(n+n-1)):
        model.append(symbolic_evolution.set_parameters(symbolic_exp[i],W,J,L,t))
        guessed.append(symbolic_evolution.set_parameters(symbolic_exp[i], guessed_W, guessed_J, guessed_decay,t))


# plt.plot(data, model_func(delay,guessed_decay,guessed_W,guessed_J), 'o')
plt.plot(data, model, 'o', label="model")
plt.plot(data, guessed, 'o', label="guessed")
plt.plot(np.linspace(-0.5,0.5,40), np.linspace(-0.5,0.5,40), 'r')
plt.grid()
plt.legend()
# print(z_points - model_func(delay,L,W,J))

KeyboardInterrupt: 